<a href="https://colab.research.google.com/github/anaconda121/Deep-Learning/blob/main/Tensorflow/Regression_Mini_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# required
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

# optional
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [2]:
tf.random.set_seed(2830)
np.random.seed(2830)

## Section 2

### Part a

In [3]:
gt1_X = np.random.randn(5000, 10)

### Part b

In [17]:
gt1_formula = np.array([5, 7, -2, -8, 6, 0, 0, 0, 0, 0])

### Part c

In [14]:
gt1_y =  gt1_X @ gt1_formula

### Final check

In [46]:
gt1_data = np.concatenate((gt1_y.reshape(len(gt1_y), 1), gt1_X), axis = 1)

In [47]:
gt1 = pd.DataFrame(gt1_data, columns = ["y", "X1", "X2", "X3", "X4", "X5", "X6", "X7", "X8", "X9", "X10"])

In [48]:
gt1.head()

,y,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10
0,-29.579884,0.110259,-3.087969,-0.749043,0.741277,-0.680544,1.606849,-0.886237,0.350773,-0.660441,-0.478209
1,-13.763301,-1.569348,-0.457992,-0.842859,-0.553458,-1.470666,0.018171,-0.660271,0.032087,0.230016,0.476304
2,-13.960092,0.509020,-0.130932,-0.276566,0.750972,-1.689004,0.895822,-1.195788,-1.034214,-0.682326,-0.010970
3,15.574768,0.188936,0.191017,-0.021699,-0.974294,0.909203,-0.436864,-0.062763,0.452257,0.365630,2.221494
4,-14.039647,-0.352854,-1.367789,-0.502583,-0.130065,-0.791091,1.052765,-0.617383,1.138698,0.763146,0.712855


## Section 3

### Part a

In [62]:
gt1_model1 = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(1,10)),
  tf.keras.layers.Dense(16, activation='relu'),
  tf.keras.layers.Dense(16, activation='relu'),
  tf.keras.layers.Dense(1, activation='linear'),
])

### Part b

In [69]:
gt1_model1.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-2), loss=tf.keras.losses.MSE, metrics=['accuracy'])

### Part c

In [71]:
gt1_model1.fit(gt1_X.reshape((len(gt1_X), 1, 10)), gt1_y, batch_size = 64, epochs=100, verbose=2)

Epoch 1/100
79/79 - 0s - loss: 0.0056 - accuracy: 0.0000e+00 - 89ms/epoch - 1ms/step
Epoch 2/100
79/79 - 0s - loss: 2.5220e-04 - accuracy: 0.0000e+00 - 87ms/epoch - 1ms/step
Epoch 3/100
79/79 - 0s - loss: 3.6541e-05 - accuracy: 0.0000e+00 - 124ms/epoch - 2ms/step
Epoch 4/100
79/79 - 0s - loss: 5.0337e-05 - accuracy: 0.0000e+00 - 101ms/epoch - 1ms/step
Epoch 5/100
79/79 - 0s - loss: 2.8968e-05 - accuracy: 0.0000e+00 - 109ms/epoch - 1ms/step
Epoch 6/100
79/79 - 0s - loss: 6.3374e-05 - accuracy: 0.0000e+00 - 112ms/epoch - 1ms/step
Epoch 7/100
79/79 - 0s - loss: 6.6898e-05 - accuracy: 0.0000e+00 - 99ms/epoch - 1ms/step
Epoch 8/100
79/79 - 0s - loss: 3.9314e-05 - accuracy: 0.0000e+00 - 108ms/epoch - 1ms/step
Epoch 9/100
79/79 - 0s - loss: 1.6923e-04 - accuracy: 0.0000e+00 - 100ms/epoch - 1ms/step
Epoch 10/100
79/79 - 0s - loss: 4.6931e-04 - accuracy: 0.0000e+00 - 117ms/epoch - 1ms/step
Epoch 11/100
79/79 - 0s - loss: 0.0016 - accuracy: 0.0000e+00 - 102ms/epoch - 1ms/step
Epoch 12/100
79/79 

In [63]:
gt1_X.reshape((len(gt1_X), 1, 10)).shape

(5000, 1, 10)